# Introduction

The aim of this challenge is to predict both weekly and daily level demand forecast

Evalualtion Metrics:
max(0,100*(1-MAPE))

# Load Libraries & Dataset

In [1]:
install.packages(c("tidygeocoder","timetk","skimr","lubridate","Metrics","plotly"))

also installing the dependencies 'hardhat', 'ipred', 'rlang', 'recipes', 'ggplot2', 'scales'




  There are binary versions available but the source versions are later:
             binary source needs_compilation
hardhat       0.1.5  0.2.0             FALSE
ipred        0.9-11 0.9-12              TRUE
rlang        0.4.11  1.0.2              TRUE
recipes      0.1.16  0.2.0             FALSE
ggplot2       3.3.3  3.3.5             FALSE
scales        1.1.1  1.2.0             FALSE
tidygeocoder  1.0.3  1.0.5             FALSE
timetk        2.6.1  2.8.0             FALSE
skimr         2.1.3  2.1.4             FALSE
lubridate    1.7.10  1.8.0              TRUE
plotly        4.9.3 4.10.0             FALSE

  Binaries will be installed
package 'ipred' successfully unpacked and MD5 sums checked
package 'rlang' successfully unpacked and MD5 sums checked
package 'lubridate' successfully unpacked and MD5 sums checked
package 'Metrics' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ser\AppData\Local\Temp\RtmpC8ZoFq\downloaded_packages


installing the source packages 'hardhat', 'recipes', 'ggplot2', 'scales', 'tidygeocoder', 'timetk', 'skimr', 'plotly'

Warning message in install.packages(c("tidygeocoder", "timetk", "skimr", "lubridate", :
"installation of package 'scales' had non-zero exit status"Warning message in install.packages(c("tidygeocoder", "timetk", "skimr", "lubridate", :
"installation of package 'recipes' had non-zero exit status"Warning message in install.packages(c("tidygeocoder", "timetk", "skimr", "lubridate", :
"installation of package 'ggplot2' had non-zero exit status"Warning message in install.packages(c("tidygeocoder", "timetk", "skimr", "lubridate", :
"installation of package 'plotly' had non-zero exit status"Warning message in install.packages(c("tidygeocoder", "timetk", "skimr", "lubridate", :
"installation of package 'timetk' had non-zero exit status"

In [2]:
library(tidyverse)
library(lubridate)
library(skimr)
library(timetk)
library(tidygeocoder)
library(catboost)
library(prophet)
library(Metrics)
library(rsample)
library(plotly)

train <- read_csv('../input/tredence-data-scientist-hiring-challenge//dataset//train.csv')
test <- read_csv('../input/tredence-data-scientist-hiring-challenge//dataset//test.csv')
sample_sub <- read_csv('../input/tredence-data-scientist-hiring-challenge//dataset//sample_submission.csv')
weekly_sub <- read_csv('../input/tredence-data-scientist-hiring-challenge//dataset//submission_weekly.csv')

Warning message:
"package 'tidyverse' was built under R version 3.6.3"

ERROR: Error: package or namespace load failed for 'tidyverse' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called 'broom'


In [3]:
set.seed(1234)

In [4]:
skim(train)

We see 10 different warehouse thats have 2 types of products. We can use warehouse id and product type as unique key to perform univariate time series

We also see missing values for the following features
* Latitude & Longitude - We can fill the missing values as location doesnt change for each warehouse
* is_weekend - we can fill them based on the date feature (Sat/Sun)
* is_warehouse_closed - extropolate from last year data
*  weekly_dispatch_count - ignore 

# Exploratory Data Analysis

Visualizing Daily level and monthly level seasonality

In [5]:
tr <- train %>% 
  unite(.,"WH_PT",c("warehouse_ID","Product_Type"),sep = "_",remove = FALSE) %>%
  group_by(WH_PT) %>% mutate(GRP_ID = cur_group_id()) %>% ungroup()

tr <- tr %>% arrange(WH_PT,date) %>%
  mutate(WK_NUM = as.integer(format(date, '%V')))
options(repr.plot.width = 14, repr.plot.height = 14)

## Daily Level - Overall

In [ ]:
daily_lvl <- tr %>% mutate(dday = day(date)) %>%
  group_by(WH_PT,GRP_ID,year,dday) %>%
  summarise(avg_qty = mean(daily_dispatch_count))

daily_lvl %>% filter(GRP_ID %in% c(1:6)) %>% ggplot(aes(year,dday))+
  geom_tile(aes(fill=avg_qty),color='white',na.rm=TRUE)+
  scale_fill_gradient(low ="yellow",high = "red")+
  facet_wrap(~WH_PT, scales='free')

In [ ]:
daily_lvl %>% filter(GRP_ID %in% c(7:12)) %>% ggplot(aes(year,dday))+
  geom_tile(aes(fill=avg_qty),color='white',na.rm=TRUE)+
  scale_fill_gradient(low ="yellow",high = "red")+
  facet_wrap(~WH_PT, scales='free')

In [ ]:
daily_lvl %>% filter(GRP_ID %in% c(13:20)) %>% ggplot(aes(year,dday))+
  geom_tile(aes(fill=avg_qty),color='white',na.rm=TRUE)+
  scale_fill_gradient(low ="yellow",high = "red")+
  facet_wrap(~WH_PT, scales='free',ncol = 2)

## Monthly Level - Overall

In [ ]:
monthly_lvl <- tr %>% mutate(dmon = month(date)) %>%
  group_by(WH_PT,GRP_ID,year,dmon) %>%
  summarise(avg_qty = mean(daily_dispatch_count))

monthly_lvl %>% filter(GRP_ID %in% c(1:6)) %>% ggplot(aes(year,dmon))+
  geom_tile(aes(fill=avg_qty),color='white',na.rm=TRUE)+
  scale_fill_gradient(low ="yellow",high = "red")+
  facet_wrap(~WH_PT, scales='free')

In [ ]:
monthly_lvl %>% filter(GRP_ID %in% c(7:12)) %>% ggplot(aes(year,dmon))+
  geom_tile(aes(fill=avg_qty),color='white',na.rm=TRUE)+
  scale_fill_gradient(low ="yellow",high = "red")+
  facet_wrap(~WH_PT, scales='free')

In [ ]:
monthly_lvl %>% filter(GRP_ID %in% c(13:20)) %>% ggplot(aes(year,dmon))+
  geom_tile(aes(fill=avg_qty),color='white',na.rm=TRUE)+
  scale_fill_gradient(low ="yellow",high = "red")+
  facet_wrap(~WH_PT, scales='free')


In [ ]:
tr %>% 
plot_ly(x=~date, y=~daily_dispatch_count, group=~WH_PT,
                           type="scatter",color=~WH_PT, mode="lines+markers")

We find
* Yearly seasonality during the pre covid era (before Mar 2020)
* A huge drop in avg demand after march & highly fluctuating, The fluctuations might b due to the effects of lockdown
* Demand for product type A is higher than the product type B 

In [ ]:
tr %>% mutate(month = lubridate::month(date,label = TRUE)) %>% 
filter(GRP_ID %in% c(1:4)&year>=2019) %>% 
  ggplot(aes(month,daily_dispatch_count,fill=factor(year)))+
  geom_boxplot()+facet_wrap(~WH_PT,nrow = 2)

## Feature Engineering 

Here we will be fixing the existing feature, treating missing values and building new features. We will be creating all necessary features for both **Machine learning** and **statistical time series models** and use them accordingly.

1. WH_PT - combine warehouse id and product type to create a unique key use to train univariate time series
2. is_weekend - (0/1) extract the binary feature from date variable
3. is_warehouse_closed - (0/1) we extrapolate from the previous year data
4. Latitude & Longitude - its repeated values for each warehouse, fill data
    * state & county - extract state and county using the latitude and longitude data
5. Time based features - extract year, month, quarter, day, day of month, week number etc
6. Fourier series - augument fourier series data for 7,30,365 days
7. n_idx - index for each warehouse product type group. this helps to capture trend when using tree based models
8. Lag values - augument lag values for 7,30,365 days
9. c19_season - (0/1) flag that differetiated pre and post covid season
10. US Holiday flag - (0/1) flag for US holidays
11. cap & floor - use to cap & floor forecast. (Used for prophet logistic growth) 

In [6]:
# Extracting State and County using lat & long 
wh_loc <- train %>% select(warehouse_ID,Latitude,Longitude) %>% distinct() %>%
  drop_na()

wh_loc <- wh_loc %>% reverse_geocode(lat = Latitude, long = Longitude)

wh_loc <- wh_loc %>% mutate(states = str_replace(word(address,-4),',',''),
                            county = str_replace(word(address,-6),',','')) %>%
  select(-Latitude,-Longitude,-address)

wh_loc

In [7]:
train$is_train <- 1
test$is_train <- 0
test$daily_dispatch_count <- NA
test$weekly_dispatch_count <- NA

full_data <- rbind(train,test)

In [9]:
#' Combining both warehouse id and product type to unique key
full_data <- full_data %>% 
  unite(.,"WH_PT",c("warehouse_ID","Product_Type"),sep = "_",remove = FALSE) %>%
  group_by(WH_PT) %>% mutate(GRP_ID = cur_group_id())

In [10]:
full_data <- full_data %>% 
  mutate(is_weekend = if_else(wday(date,label = TRUE) %in% c("Sun","Sat"),1,0))

In [11]:
#' replacing missing values of warehouse closed based on the last year lag
full_data <- full_data %>% group_by(WH_PT) %>%
  mutate(is_warehouse_lag = lag(is_warehouse_closed,n=365,default="SKIP"))

full_data %>% group_by(is_warehouse_closed,is_warehouse_lag) %>% count() %>%
  filter(is_warehouse_lag!='SKIP')


full_data <- full_data %>% mutate(is_warehouse_closed = if_else(is.na(is_warehouse_closed),
                                                                is_warehouse_lag,
                                                                is_warehouse_closed)) %>%
  mutate(is_wh_cls = if_else(is_warehouse_closed=="Yes",1,0)) %>%
  select(-is_warehouse_closed,-is_warehouse_lag)


In [12]:
#' Filling the missing values in both Latitude and Longitude
#' By filling the above or below value for the latitude and longitude
full_data <- full_data %>% fill(Latitude,.direction = 'downup') %>%
  fill(Longitude,.direction = 'downup')

In [13]:
#' Merging warehouse location data 
full_data <- full_data %>% left_join(.,wh_loc,by="warehouse_ID")

#' Creating a covid-19 flag based on the first lockdown initiated
#' renaming date to ds and daily dispatch to y
full_data <- full_data %>% ungroup()  %>%
  rename(c("ds"="date","y"="daily_dispatch_count")) %>%
  arrange(WH_PT,ds) %>%
  mutate(c19_season = if_else(ds > '2020-03-15',1,0))

In [14]:

#' Augumenting US holidays flag
full_data <- full_data %>% tk_augment_holiday_signature(.date_var = ds,
                                                        .holiday_pattern = "none",
                                                        .locale_set = "US",
                                                        .exchange_set = "US")


In [15]:
#' Extract time based features
#' 
full_data <- full_data %>% ungroup() %>% 
  mutate(year= lubridate::year(ds),
         month = lubridate::month(ds,label = TRUE),
         n_quarter = lubridate::quarter(ds),
         n_day = lubridate::day(ds),
         n_dom = lubridate::days_in_month(ds),
         n_week = lubridate::isoweek(ds),
         n_wday = lubridate::wday(ds,label = TRUE)) %>% 
  tk_augment_fourier(.date_var = ds,.periods = c(7,30,365),.K=1)

In [16]:
#' Create floor and Cap values at each group level
#' create trend component for tree based models
#' Crete Lag values
full_data <- full_data %>%
  group_by(WH_PT) %>%
  # for logistic growth of prophet
  mutate(floor = min(y,na.rm = T),cap=max(y,na.rm = T)*1.25) %>%
  mutate(n_idx = row_number(),n_idx_sqr = row_number()^2) %>%
  arrange(WH_PT,ds)%>% 
  tk_augment_lags(y,.lags = c(7,30,365)) %>%
  ungroup()

In [17]:
skim(full_data)

## Model Training 

When it comes to modeling time series we have use both Statistical Model(ARIMA, ETS) and Machine Learning Models(Boosted Trees). 

From below table of checking historical data, we see the time period for each warehouse product type is different. Traditional models like ARIMA and ETS requires a full cycle fo series, which means it will not work on short time series. Inorder to over come this issue I have consider Facebook's Prophet model for univariate modeling. 

For Machine Learning based time series model, I am considering both Catboost and H2O AutoML

#### Models Experiemented
Here are the list of models that I have experimented during the course of this hackathon. Note that I will adding very few models here in this notebook. Please find the rest of the models in each R file.

List of models exprimented
1. **Baseline Prophet Model**- Used xreg features is_weekend, is_wh_cls, c19_season, locale_US
2. **Baseline Catboost** - Used all xreg including fourier, time, holiday based features 
3. **Prophet+Catboost** - Ran vanilla prophet, extracted its residual errors and used it as outcome to catboost model with xreg features (yhat= Prophet_prediction + catboost_residual_prediction)
4. **Prophet(Logistic)+Catboost** - Same as above model, used logistic growth for prophet
5. **H2O AutoML** - Ran a baseline h2o automl model (similar to catboost), Also a variant, where we cap forecast max(0,forecast)
6. **AutoRegressor Catboost** - Ran individual catboost model to each warehouse product type, used lag features and ran a recursive auto regressive model to consider new forecast values as lag values for future prediction


In [18]:
#' Create train and test data for ML models
tr_data <- full_data %>% filter(is_train==1) %>%
  mutate_if(is.factor,as.numeric) %>%
  mutate_if(is.character,as.factor) %>%
  mutate_at(all_of("year"),factor) %>%
  select(-GRP_ID) %>%
  replace_na(list(y_lag365=0,y_lag7=0,y_lag30=0))

In [19]:
te_data <- full_data %>% filter(is_train==0)%>%
  mutate_if(is.factor,as.numeric) %>%
  mutate_if(is.character,as.factor) %>%
  mutate_at(all_of("year"),factor) %>%
  select(-GRP_ID) %>%
  replace_na(list(y_lag365=0))

In [56]:
#' extract validation set
val_data <- tr_data %>% filter(is_train==1&year==2020&n_week %in% c(41,42,43,44,45,46,47,48,49,50))

In [21]:
wkly_dispatch_check <- full_data %>% group_by(WH_PT,year,n_week) %>%
  mutate(wkly_dispatch = sum(y)) %>% drop_na() %>% select(ds,WH_PT,wkly_dispatch)
head(wkly_dispatch_check)

In [24]:
#' Use this data for all prophet models
hist_data <- full_data %>% filter(is_train==1) %>%
  select(WH_PT,ds,y,is_weekend,is_wh_cls,c19_season,locale_US,cap,floor)
fut_data <- full_data %>% filter(is_train==0)%>%
  select(WH_PT,ds,is_weekend,is_wh_cls,c19_season,locale_US,cap,floor)

In [25]:
hist_data %>% group_by(WH_PT) %>% count()

In [26]:
unique_wh <- full_data %>% distinct(WH_PT) %>% pull(WH_PT)

xreg_cols <- colnames(hist_data)[c(4:7)]

hist_fit <- tibble(NULL)
future_frcst <- tibble(NULL)
xreg_cols
options( warn = -1 )

In [27]:
#' run the for loop for prophet using xreg values
for(wh in unique_wh){
  ## Prophet
  message('====== Running prophet for ',wh)
  
  df_x <- hist_data %>% filter(WH_PT==wh) %>%
    select(-WH_PT)
  df_y <- fut_data %>% filter(WH_PT==wh) %>%
    select(-WH_PT)
  
  fit_prophet <- prophet(growth='logistic')
  
  if(!is.null(xreg_cols)){
    for(xr in xreg_cols){
      fit_prophet <- add_regressor(fit_prophet,xr)
    }
  }
  
  fit_prophet <- fit.prophet(fit_prophet,df_x)
  
  prophet_fit <- stats::predict(fit_prophet,df_x) %>% pull(yhat)
  prophet_pred <- stats::predict(fit_prophet,df_y) %>% pull(yhat)
    
  # message('Running arima for ',wh)
  # # run arima
  # ts_x <- ts(df_x$y,frequency = 365)
  # fcst_horizon <- nrow(df_y)
  # 
  # if(!is.null(xreg_cols)){
  #   xreg <- df_x %>% select(all_of(xreg_cols)) %>% as.matrix()
  #   yreg <- df_y %>% select(all_of(xreg_cols)) %>% as.matrix()
  # }else{
  #   xreg <- NULL
  #   yreg <- NULL
  # }
  # 
  # fit_autoar <- auto.arima(ts_x,xreg = xreg)
  # 
  # autoar_fit <- pmax(as.numeric(fit_autoar$fitted),0)#since its giving -ve
  # autoar_pred <- pmax(as.numeric(forecast(fit_autoar,xreg=yreg)$mean),0)
  
  
  # message('Running HoltsWinter for ',wh)
  # #run holts
  # fit_holtswin <- stats::HoltWinters(ts_x)
  # 
  # holts_fit <- as.numeric(fit_holtswin$fitted[,'xhat'])
  # holts_pred <- as.numeric(forecast(fit_holtswin, h = nrow(yreg))$mean)
  
  tr_fit <- tibble(ds=df_x$ds,actual=df_x$y,
                   yhat=prophet_fit,
                   #auto_arima = autoar_fit,
                   #holtswint = holts_fit,
                   WH_PT=wh,
                   model = 'Prophet+is_wknd_is_wh+c19+US')
  
  fut_fit <- tibble(ds=df_y$ds,
                    yhat=prophet_pred,
                    #auto_arima = autoar_pred,
                    #holtswint = holts_pred,
                    WH_PT=wh,
                    model = 'Prophet+is_wknd_is_wh+c19+US')
  hist_fit <- rbind(hist_fit,tr_fit)
  future_frcst <- rbind(future_frcst,fut_fit)
}

#### Bayesian Optimized Catboost Model 

In [28]:
colnames(tr_data)

In [29]:
xreg_catboost <- tr_data %>% select(contains('ds_'),contains('n_'),contains('y_lag'),
                                   all_of(c('county','states','c19_season','locale_US','is_weekend','is_wh_cls','warehouse_ID',
                                   'Product_Type','year','month'))) %>% colnames()
y <- 'y'
xreg_catboost

In [30]:
str(tr_data)

In [31]:
tr_pool <- catboost.load_pool(data = tr_data[,xreg_catboost],
                              label = unlist(tr_data[,y]))

te_pool <- catboost.load_pool(data = te_data[,xreg_catboost])

va_pool <- catboost.load_pool(data = val_data[,xreg_catboost],
                             label = unlist(val_data[,y]))

In [32]:
install.packages('rBayesianOptimization')
library(rBayesianOptimization)

In [33]:
#' Grid for bayesan
grid_search <- expand.grid(
  iterations = c(1000L,500L),
  depth = c(4L,5L,6L),
  learning_rate = c(0.01,0.03)
)

grid_bound = list(
  iterations = c(1000L,500L),
  depth = c(4L,6L),
  learning_rate = c(0.01,1L))

In [38]:
catboost_fit <- function(iterations,depth,learning_rate){
  
  model <- catboost.train(tr_pool,
                          va_pool,
                          params = list(
                            iterations=iterations,
                            depth = depth,
                            learning_rate = learning_rate,
                            loss_function="RMSE",
                            eval_metric='MAPE',
                            random_seed = 55,
                            l2_leaf_reg=8,
                            langevin=TRUE,
                            early_stopping_rounds=200,
                            od_type='Iter',
                            metric_period = 50,
                            od_wait=20,
                            rsm=0.5,
                            use_best_model=TRUE)
  )
  
  val_enum <- catboost.predict(model,va_pool)
  error <- rmse(val_data$y,val_enum)
  
  result <- list(Score = error,Pred=0)
  
  return(result)
}

In [39]:
set.seed(1234)

bayes_cat <- BayesianOptimization(FUN = catboost_fit,bounds = grid_bound,
                                  init_points = 0,init_grid_dt = grid_search,
                                  n_iter = 10,acq = "ei",verbose = TRUE)

In [40]:
bayes_cat$Best_Par

In [41]:
model_train <- catboost.train(tr_pool,params = list(
  depth=bayes_cat$Best_Par['depth'],
  iterations = bayes_cat$Best_Par['iterations'],
  learning_rate=bayes_cat$Best_Par['learning_rate'],
  loss_function="RMSE",
  eval_metric = "MAPE",
  od_pval = 10^-5,
  use_best_model=TRUE,
  metric_period=100
  ))

In [42]:
feat_imp <- as_tibble(catboost.get_feature_importance(model_train))%>% 
  rownames_to_column() %>% cbind(.,xreg_catboost) %>% arrange(desc(V1))
feat_imp

In [58]:
#' Validation prediction and test prediction

val_prediction <- catboost.predict(model_train,va_pool)

te_prediction <- catboost.predict(model_train,te_pool)

#' we saw negative values in the prediction, hence flooring the prediction at 0
val_daily_cat <- val_data %>% select(ds,WH_PT,n_week,year,y) %>% mutate(y_hat = pmax(val_prediction,0))
head(val_daily_cat)

In [51]:
options(repr.plot.width = 14, repr.plot.height = 8)
hist_samp <- sample(unique_wh,1)
val_daily_cat %>% filter(WH_PT==hist_samp) %>%
  ggplot(aes(ds))+
  geom_line(aes(y=y),color="red")+
  geom_line(aes(y=y_hat),color="steelblue")+
labs(title=hist_samp)

we can see that the model had fit perfectly. But what are the chances that its over fit we are not sure unless we test it in the open 

In [52]:
# Predict the weekly data
val_cat_weekly_ <- val_daily_cat %>% 
    mutate(n_week=lubridate::isoweek(ds),
          year=year(ds)) %>% 
    group_by(year,WH_PT,n_week) %>% summarise(yhat=sum(y_hat))
val_cat_weekly <- val_cat_weekly_ %>%
inner_join(.,wkly_dispatch_check,by=c("year","WH_PT","n_week"))
head(val_cat_weekly)

In [53]:
hist_samp <- sample(unique_wh,1)
val_cat_weekly %>% filter(WH_PT==hist_samp) %>%
  ggplot(aes(ds))+
  geom_line(aes(y=wkly_dispatch),color="red")+
  geom_line(aes(y=yhat),color="steelblue")+
labs(title=hist_samp)

In [55]:
tr_data %>% filter(WH_PT=='WH_0x3f2_Type_A')

### Catboost Bayesian Submission

In [86]:
#' Create weekly forecast using the daily forecast
#' Lets group by the WH_PT key and the week number,
#' as we know that the sum of each weeks daily forecast = weekly forecast

weekly_sub <- weekly_sub %>% mutate(flg=1)

test_data <- full_data %>% filter(is_train==0) %>%
  select(ID,ds,WH_PT) %>% left_join(.,weekly_sub,by="ID") %>%
  mutate(WK_NBR = as.integer(format(ds, '%V'))) %>%
  mutate(cat=pmax(te_prediction,0))


weekly_submission <- test_data %>% group_by(WH_PT,WK_NBR) %>%
  mutate(weekly_dispatch_count = sum(cat))


weekly_sub_final <- weekly_submission %>% filter(flg==1) %>% ungroup() %>%
  select(ID,weekly_dispatch_count)



In [89]:
#" Final submission

filename <- paste('ak_catboost_bayes_3882_6_0-29',format(Sys.time(),"%Y%m%d%H%M%s"),sep = '_')
write.csv(weekly_sub_final,paste0(filename,'.csv',collapse = ''),row.names = FALSE)

## Baseline Prophet - Validation

Now lets validate the forecast at daily level for the above model

In [84]:
options(repr.plot.width = 14, repr.plot.height = 8)
hist_samp <- sample(unique_wh,1)
val_daily_cat <- hist_fit %>% filter(ds %in% val_data$ds)
val_daily_dispatch %>% filter(WH_PT==hist_samp) %>%
  ggplot(aes(ds))+
  geom_line(aes(y=actual),color="red")+
  geom_line(aes(y=prophet),color="steelblue")+
labs(title=hist_samp)

Now lets validate the forecast at weekly level

In [ ]:
wkly_dispatch_check <- full_data %>% group_by(WH_PT,year,n_week) %>%
  mutate(wkly_dispatch = sum(y)) %>% drop_na() %>% select(ds,WH_PT,wkly_dispatch)
head(wkly_dispatch_check)

In [ ]:
val_weekly_dispatch <- val_daily_dispatch %>% 
    mutate(n_week=lubridate::isoweek(ds),
          year=year(ds)) %>% 
    group_by(year,WH_PT,n_week) %>% summarise(yhat=sum(prophet))
val_weekly <- val_weekly_dispatch %>%
inner_join(.,wkly_dispatch_check,by=c("year","WH_PT","n_week"))
head(val_weekly)

In [ ]:
hist_samp <- sample(unique_wh,1)
val_weekly %>% filter(WH_PT==hist_samp) %>%
  ggplot(aes(ds))+
  geom_line(aes(y=wkly_dispatch),color="red")+
  geom_line(aes(y=yhat),color="steelblue")+
labs(title=hist_samp)

## Submission of Model

In [ ]:
#' Create weekly forecast using the daily forecast
#' Lets group by the WH_PT key and the week number,
#' as we know that the sum of each weeks daily forecast = weekly forecast

weekly_sub <- weekly_sub %>% mutate(flg=1)

test_data <- full_data %>% filter(is_train==0) %>%
  select(ID,ds,WH_PT) %>% left_join(.,weekly_sub,by="ID") %>%
  mutate(WK_NBR = as.integer(format(ds, '%V'))) %>%
  left_join(.,future_frcst,by=c("ds"="df_y$ds","WH_PT")) %>%
  mutate(prophet=pmax(prophet,0))


weekly_submission <- test_data %>% group_by(WH_PT,WK_NBR) %>%
  mutate(weekly_dispatch_count = sum(prophet))


weekly_sub_final <- weekly_submission %>% filter(flg==1) %>% ungroup() %>%
  select(ID,weekly_dispatch_count)

#" Final submission

filename <- paste('ak_prophet',format(Sys.time(),"%Y%m%d%H%M%s"),sep = '_')
write.csv(weekly_sub_final,paste0(filename,'.csv',collapse = ''),row.names = FALSE)

write.csv(hist_fit,'prophet_hist_data_actual_pred.csv',row.names = FALSE)